# Hybrid RAG Chatbot: LangChain + Direct OpenAI API with Memory

## Learning Objectives
In this notebook, you'll learn:
1. **LangChain for Retrieval**: Use LangChain for web scraping, document processing, embeddings, and similarity search
2. **Direct OpenAI API**: Use OpenAI's `responses.create()` endpoint directly for more control
3. **Conversation Memory**: Implement custom conversation memory without LangChain's memory classes
4. **Hybrid Architecture**: Understand when to use each tool and how to integrate them

## Architecture Overview
```
User Question
     ↓
[LangChain] → Similarity Search → Retrieved Documents
     ↓
[OpenAI API] → responses.create() with Memory → Answer
     ↓
Update Conversation History
```

---
## Part 1: Setup and Installation
Install all required packages

In [1]:
# Install required packages
!pip install -qU langchain langchain-core langchain-openai langchain-community chromadb beautifulsoup4 openai


[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


---
## Part 2: Initialize OpenAI Client

**Important Security Note**: Never hardcode API keys in production code.
Use environment variables or secure vaults instead.

In [ ]:
import os
from openai import OpenAI

# SECURITY WARNING: Use environment variables for API keys
# Load your API key from .env file: OPENAI_API_KEY=your_key_here
openai_api_key = os.getenv('OPENAI_API_KEY')  # Set this in your .env file

# Initialize the OpenAI client
# This client will be used for direct API calls later
client = OpenAI(api_key=openai_api_key)

print("✅ OpenAI client initialized successfully")

✅ OpenAI client initialized successfully


---
## Part 3: Data Collection with LangChain

### Step 3.1: Define URLs to Scrape
We'll use Wikipedia's Taj Mahal article as our knowledge source

In [3]:
import requests
from bs4 import BeautifulSoup

# Define the URL(s) to scrape
urls = ['https://en.wikipedia.org/wiki/Taj_Mahal']

print(f"📚 Collected {len(urls)} URL(s) to process")
print(f"URLs: {urls}")

📚 Collected 1 URL(s) to process
URLs: ['https://en.wikipedia.org/wiki/Taj_Mahal']


### Step 3.2: Load Documents with LangChain

**What is WebBaseLoader?**
- A LangChain utility that fetches web pages and converts them to Document objects
- Handles HTTP requests and parsing automatically
- Returns structured data that's easy to process

In [4]:
from langchain_community.document_loaders import WebBaseLoader

# Create a loader with custom headers to avoid being blocked
loader = WebBaseLoader(
    urls,
    requests_kwargs={
        'headers': {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'
        }
    }
)

# Load the documents
pages = loader.load()

print(f"✅ Loaded {len(pages)} document(s)")
print(f"Document type: {type(pages[0])}")
print(f"\nFirst 500 characters:\n{pages[0].page_content[:500]}...")

USER_AGENT environment variable not set, consider setting it to identify your requests.


✅ Loaded 1 document(s)
Document type: <class 'langchain_core.documents.base.Document'>

First 500 characters:




Taj Mahal - Wikipedia





























Jump to content







Main menu





Main menu
move to sidebar
hide



		Navigation
	


Main pageContentsCurrent eventsRandom articleAbout WikipediaContact us





		Contribute
	


HelpLearn to editCommunity portalRecent changesUpload fileSpecial pages



















Search











Search






















Appearance
















Donate

Create account

Log in








Personal tools





Donate Create account Log in










...


### Step 3.3: Document Statistics
Let's analyze the scraped data before processing

In [5]:
# Calculate character counts for each page
char_counts = [len(d.page_content or "") for d in pages]

print("📊 Document Statistics:")
print(f"Total pages: {len(pages)}")
print(f"Total characters: {sum(char_counts):,}")
print(f"Average chars/page: {round(sum(char_counts) / max(1, len(char_counts)), 2):,}")
print(f"Min/Max chars: {min(char_counts, default=0):,} / {max(char_counts, default=0):,}")

📊 Document Statistics:
Total pages: 1
Total characters: 66,994
Average chars/page: 66,994.0
Min/Max chars: 66,994 / 66,994


---
## Part 4: Data Preprocessing

### Step 4.1: Normalize Text

**Why normalize?**
- Web-scraped text often contains irregular whitespace, HTML entities, and special characters
- Normalization improves embedding quality and retrieval accuracy
- Reduces noise in the vector database

In [6]:
import re
import html
from langchain_core.documents import Document

def normalize_doc(doc: Document) -> Document:
    """
    Normalize a document by cleaning up whitespace and special characters.
    
    Steps:
    1. Unescape HTML entities (e.g., &nbsp; → space)
    2. Replace non-breaking spaces with regular spaces
    3. Remove zero-width characters
    4. Collapse multiple spaces/tabs into single space
    5. Limit consecutive newlines to maximum of 2
    6. Trim whitespace around newlines
    
    Args:
        doc: LangChain Document object
    
    Returns:
        Cleaned Document object with same metadata
    """
    text = doc.page_content or ""
    
    # Step 1: Unescape HTML entities
    text = html.unescape(text)
    
    # Step 2: Replace non-breaking spaces
    text = text.replace("\xa0", " ")
    
    # Step 3: Remove zero-width characters
    text = re.sub(r"[\u200B-\u200D\uFEFF]", "", text)
    
    # Step 4: Collapse spaces and tabs
    text = re.sub(r"[ \t]+", " ", text)
    
    # Step 5: Limit blank lines
    text = re.sub(r"\n{3,}", "\n\n", text)
    
    # Step 6: Trim around newlines
    text = re.sub(r" *\n *", "\n", text).strip()
    
    return Document(page_content=text, metadata=doc.metadata)

# Apply normalization to all documents
clean_docs = [normalize_doc(d) for d in pages]

print("✅ Documents normalized")
print(f"\nCleaned text preview (first 500 chars):\n{clean_docs[0].page_content[:500]}...")

✅ Documents normalized

Cleaned text preview (first 500 chars):
Taj Mahal - Wikipedia

Jump to content

Main menu

Main menu
move to sidebar
hide

Navigation


Main pageContentsCurrent eventsRandom articleAbout WikipediaContact us

Contribute


HelpLearn to editCommunity portalRecent changesUpload fileSpecial pages

Search

Search

Appearance

Donate

Create account

Log in

Personal tools

Donate Create account Log in

Contents
move to sidebar
hide

(Top)

1
Etymology

2
Inspiration

3
Architecture and design

Toggle Architecture and design subsection

3.1
...


### Step 4.2: Split Documents into Chunks

**Why chunk documents?**
- Embedding models have token limits (typically 8,192 tokens)
- Smaller chunks = more precise retrieval
- Overlap ensures context isn't lost at chunk boundaries

**Parameters explained:**
- `chunk_size=2000`: Each chunk will be ~2000 characters
- `chunk_overlap=150`: 150 characters overlap between chunks to maintain context
- `separators`: Try to split on paragraphs first, then lines, then words, then characters

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Create a text splitter with sensible defaults
splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,           # Maximum characters per chunk
    chunk_overlap=150,          # Overlap to maintain context
    separators=["\n\n", "\n", " ", ""],  # Try these separators in order
)

# Split documents into chunks
chunks = splitter.split_documents(clean_docs)

print(f"✅ Created {len(chunks)} chunks from {len(clean_docs)} document(s)")
print(f"\nExample chunk:\n{'-' * 80}\n{chunks[0].page_content}\n{'-' * 80}")

✅ Created 50 chunks from 1 document(s)

Example chunk:
--------------------------------------------------------------------------------
Taj Mahal - Wikipedia

Jump to content

Main menu

Main menu
move to sidebar
hide

Navigation


Main pageContentsCurrent eventsRandom articleAbout WikipediaContact us

Contribute


HelpLearn to editCommunity portalRecent changesUpload fileSpecial pages

Search

Search

Appearance

Donate

Create account

Log in

Personal tools

Donate Create account Log in

Contents
move to sidebar
hide

(Top)

1
Etymology

2
Inspiration

3
Architecture and design

Toggle Architecture and design subsection

3.1
Exterior

3.2
Interior

3.3
Garden

3.4
Outlying buildings

4
Construction

5
Later years

6
Symbolism

7
Tourism

8
Myths

9
See also

10
Notes

11
References

Toggle References subsection

11.1
Citations

11.2
General sources

12
External links

Toggle the table of contents

Taj Mahal

168 languages
-------------------------------------------------------------

---
## Part 5: Create Vector Database with LangChain

### Step 5.1: Generate Embeddings and Store in ChromaDB

**What are embeddings?**
- Numerical representations of text that capture semantic meaning
- Similar text → similar embedding vectors
- Enables semantic search (find by meaning, not just keywords)

**What is ChromaDB?**
- Open-source vector database
- Stores embeddings and enables fast similarity search
- Runs locally without external dependencies

In [8]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
import os

# Define the directory where vector database will be stored
PERSIST_DIRECTORY = "./chroma_db"

# Initialize OpenAI embeddings
# This uses OpenAI's text-embedding-ada-002 model by default
embeddings = OpenAIEmbeddings(
    openai_api_key=openai_api_key
)

print("🔄 Creating embeddings and building vector database...")
print("This may take a minute depending on the number of chunks...")
print(f"📁 Database will be saved to: {PERSIST_DIRECTORY}")

# Create vector database from documents with persistence
# This will:
# 1. Generate embeddings for each chunk using OpenAI
# 2. Store embeddings in ChromaDB
# 3. Save to local directory for future use
# 4. Create an index for fast similarity search
vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    collection_name="taj_mahal_knowledge",
    persist_directory=PERSIST_DIRECTORY  # 👈 This saves to disk!
)

print(f"✅ Vector database created with {len(chunks)} chunks")
print(f"✅ Database persisted to: {PERSIST_DIRECTORY}")
print(f"Collection name: taj_mahal_knowledge")

# Verify the directory was created
if os.path.exists(PERSIST_DIRECTORY):
    print(f"✅ Directory created successfully")
    print(f"📊 Directory size: {sum(os.path.getsize(os.path.join(PERSIST_DIRECTORY, f)) for f in os.listdir(PERSIST_DIRECTORY) if os.path.isfile(os.path.join(PERSIST_DIRECTORY, f))) / 1024:.2f} KB")

🔄 Creating embeddings and building vector database...
This may take a minute depending on the number of chunks...
📁 Database will be saved to: ./chroma_db
✅ Vector database created with 50 chunks
✅ Database persisted to: ./chroma_db
Collection name: taj_mahal_knowledge
✅ Directory created successfully
📊 Directory size: 1280.00 KB


### Step 5.2: Create a Retriever

**What is a retriever?**
- Interface for querying the vector database
- Takes a question and returns most relevant chunks
- Uses cosine similarity to find semantically similar content

In [9]:
# Create a retriever from the vector database
# Default: returns top 4 most similar chunks
retriever = vectordb.as_retriever(
    search_kwargs={"k": 4}  # Number of chunks to retrieve
)

print("✅ Retriever created")
print("Configuration: Top 4 most relevant chunks will be retrieved")

✅ Retriever created
Configuration: Top 4 most relevant chunks will be retrieved


### Step 5.3: Test the Retriever
Let's verify that similarity search works correctly

In [10]:
def pretty_print_docs(docs):
    """Helper function to display retrieved documents in a readable format"""
    print("\n" + "=" * 100)
    for i, doc in enumerate(docs, 1):
        print(f"\n📄 Document {i}:")
        print("-" * 100)
        print(doc.page_content)
        print("-" * 100)
    print("=" * 100)

# Test query
test_question = "What materials were used in the main mausoleum and where did some of these materials originate?"

print(f"🔍 Testing retrieval with question: '{test_question}'\n")

# Retrieve relevant documents
retrieved_docs = retriever.invoke(test_question)

print(f"Found {len(retrieved_docs)} relevant chunks:")
pretty_print_docs(retrieved_docs)

🔍 Testing retrieval with question: 'What materials were used in the main mausoleum and where did some of these materials originate?'

Found 4 relevant chunks:


📄 Document 1:
----------------------------------------------------------------------------------------------------
Exterior
The eight sided main structure with large arched doorways and minarets on a square plinth
The mausoleum building is the central structure of the entire complex. It is a white marble structure standing on a 6-metre (20 ft) high square plinth with sides measuring 95.5 metres (313 ft) in length. The base structure is a large multi-chambered cube with chamfered corners forming an eight-sided structure that is approximately 57.3 metres (188 ft) long on each of the four long sides.[24]
The building has four identical sides with iwans (arch-shaped doorways), topped by a large dome and finial. Each side of the iwan is framed with a 33-metre (108 ft) high pishtaq (vaulted archway) with two similarly shaped arched b

---
## Part 6: Conversation Memory Management

### Understanding Conversation Memory

**Why do we need memory?**
- OpenAI's API is stateless (doesn't remember previous interactions)
- We need to manually track conversation history
- Memory enables context-aware conversations and follow-up questions

**Our Memory Implementation:**
1. **Storage**: List of dictionaries with `role` and `message` keys
2. **Sliding Window**: Keep only last 10 messages to avoid token overflow
3. **Formatting**: Convert history to readable text for the LLM
4. **Commands**: Support for viewing and clearing history

**Memory Functions:**
- `add_to_memory()`: Saves user/assistant messages
- `get_memory_text()`: Formats history as string
- `clear_memory()`: Resets conversation state

In [ ]:
# Initialize conversation memory as a list
# Each entry is a dictionary with 'role' and 'message' keys
conversation_history = []

def add_to_memory(role, message):
    """
    Save a message to conversation memory.
    
    Args:
        role (str): Either 'user' or 'assistant'
        message (str): The actual message content
    
    Implements sliding window: keeps only last 10 messages
    to prevent context overflow and manage token limits.
    """
    conversation_history.append({"role": role, "message": message})
    
    # Keep only last 10 messages (5 exchanges)
    if len(conversation_history) > 10:
        conversation_history.pop(0)  # Remove oldest message

def get_memory_text():
    """
    Format conversation history as readable text.
    
    Returns:
        str: Formatted conversation history or default message
    
    Format: "user: question\nassistant: answer\n..."
    This gets included in the instructions to provide context.
    """
    if not conversation_history:
        return "No previous conversation."
    
    return "\n".join([f"{msg['role']}: {msg['message']}" for msg in conversation_history])

def clear_memory():
    """
    Clear all conversation memory.
    Useful for starting fresh or when context becomes confusing.
    """
    conversation_history.clear()
    print("🧹 Memory cleared!")

print("✅ Memory system initialized!")
print(f"📊 Current memory size: {len(conversation_history)} messages")
print("💾 Max capacity: 10 messages (sliding window)")

✅ Memory system ready!


---
## Part 7: Direct OpenAI API Integration

### Understanding `responses.create()` Endpoint

**Key Differences from `chat.completions.create()`:**

| Feature | `responses.create()` | `chat.completions.create()` |
|---------|---------------------|----------------------------|
| **Parameters** | `input` + `instructions` | `messages` array |
| **Structure** | Simpler, two-parameter model | Structured message history |
| **System Prompt** | `instructions` parameter | First message with role="system" |
| **User Input** | `input` parameter | Messages with role="user" |
| **Memory Handling** | Manual formatting required | Array structure |
| **Use Case** | Simple Q&A, single responses | Multi-turn conversations |

**Why use `responses.create()` here?**
- Cleaner separation of context (retrieved docs) and conversation history
- Easier to template and format different types of input
- More explicit control over what goes where

**Our Implementation Strategy:**
1. Retrieve relevant documents from vector database
2. Get conversation history from memory
3. Format `instructions` with memory + system prompt
4. Format `input` with retrieved context + current question
5. Get response and extract clean text
6. Update memory with Q&A pair

---
## Part 8: Interactive Chatbot Interface

### Chatbot Features

**Commands:**
- **Regular input**: Ask any question about the Taj Mahal
- **`history`**: View entire conversation history
- **`clear`**: Reset conversation memory
- **`quit`** or **`exit`**: End the session

**How it works:**
1. User enters a question
2. System processes special commands (history/clear/quit)
3. For regular questions:
   - Retrieves relevant context from vector database
   - Includes conversation memory for context
   - Generates answer using OpenAI API
   - Saves Q&A to memory
4. Displays formatted response

**Example Conversation Flow:**
```
User: "Who built the Taj Mahal?"
Bot: "Shah Jahan built the Taj Mahal..."

User: "Why did he build it?"  ← Note: "he" requires memory!
Bot: "He built it in memory of his wife Mumtaz Mahal..."
```

The chatbot understands pronouns and references because it maintains conversation memory!

In [ ]:
def ask_question(question):
    """
    Process a question using RAG (Retrieval-Augmented Generation) with memory.
    
    This function implements the complete RAG pipeline:
    1. RETRIEVE: Get relevant context from vector database
    2. AUGMENT: Combine context with conversation memory
    3. GENERATE: Use OpenAI API to create response
    4. REMEMBER: Save interaction to memory
    
    Args:
        question (str): User's question
    
    Returns:
        str: AI-generated answer based on retrieved context and memory
    
    Architecture:
        Question → Retriever → Context
                              ↓
        Memory → Format → Instructions
                              ↓
        Context + Question → Input
                              ↓
        OpenAI API → Response → Update Memory
    """
    
    # ============================================================
    # STEP 1: RETRIEVE - Find relevant documents from vector DB
    # ============================================================
    # Use LangChain retriever to find top 4 most similar chunks
    docs = retriever.invoke(question)
    
    # Combine all retrieved documents into single context string
    # Separated by double newlines for readability
    context = "\n\n".join([doc.page_content for doc in docs])
    
    # ============================================================
    # STEP 2: GET MEMORY - Retrieve conversation history
    # ============================================================
    # Format previous Q&A pairs as text to maintain context
    memory = get_memory_text()
    
    # ============================================================
    # STEP 3: BUILD INSTRUCTIONS - System prompt + memory
    # ============================================================
    # This acts as the "system message" in traditional chat models
    # Includes:
    # - Role definition (helpful assistant about Taj Mahal)
    # - Previous conversation for context
    # - Guidelines for using both memory and retrieved context
    instructions = f"""You are a helpful assistant about the Taj Mahal.

Previous conversation:
{memory}

Use the conversation history and context to answer questions accurately.
If the user refers to previous topics (like "it", "that", "he/she"), 
use the conversation history to understand what they're referring to."""
    
    # ============================================================
    # STEP 4: BUILD INPUT - Combine retrieved context + question
    # ============================================================
    # This is the "user message" containing:
    # - Retrieved context from vector database (RAG component)
    # - Current question from user
    input_text = f"""Context from knowledge base:
{context}

Current question: {question}"""
    
    # ============================================================
    # STEP 5: CALL OPENAI RESPONSES API
    # ============================================================
    # Using responses.create() instead of chat.completions.create()
    # Parameters:
    # - model: gpt-4o-mini (fast and cost-effective)
    # - input: User query + retrieved context
    # - instructions: System prompt + conversation memory
    # - temperature: 0.7 (balanced creativity/accuracy)
    # - max_output_tokens: 500 (limit response length)
    response = client.responses.create(
        model="gpt-4o-mini",
        input=input_text,
        instructions=instructions,
        temperature=0.7,
        max_output_tokens=500
    )
    
    # ============================================================
    # STEP 6: EXTRACT CLEAN TEXT
    # ============================================================
    # response.output_text provides direct string output
    # No need to parse complex response objects
    answer = response.output_text
    
    # ============================================================
    # STEP 7: SAVE TO MEMORY - Remember this interaction
    # ============================================================
    # Add both question and answer to conversation history
    # This enables follow-up questions and contextual understanding
    add_to_memory("user", question)
    add_to_memory("assistant", answer)
    
    return answer

print("✅ RAG + Memory chat function ready!")
print("🔍 Retrieval: LangChain vector database")
print("🤖 Generation: OpenAI responses.create()")
print("💾 Memory: Custom conversation tracking")

✅ Chat function ready!


In [ ]:
# ============================================================
# INTERACTIVE CHATBOT LOOP
# ============================================================
# This loop handles user interaction and command processing

print("\n" + "="*80)
print("🎉 TAJ MAHAL CHATBOT WITH MEMORY")
print("="*80)
print("Ask me anything about the Taj Mahal!")
print("\nAvailable commands:")
print("  • quit/exit  - End the conversation")
print("  • clear      - Reset conversation memory")
print("  • history    - View conversation history")
print("="*80 + "\n")

while True:
    # Get user input
    user_input = input("👤 You: ").strip()
    
    # ========================================
    # COMMAND: Quit/Exit
    # ========================================
    if user_input.lower() in ['quit', 'exit']:
        print("\n👋 Goodbye! Thanks for chatting!\n")
        break
    
    # ========================================
    # COMMAND: Clear memory
    # ========================================
    elif user_input.lower() == 'clear':
        clear_memory()
        print("✅ You can now start a fresh conversation.\n")
        continue
    
    # ========================================
    # COMMAND: Show history
    # ========================================
    elif user_input.lower() == 'history':
        print("\n📜 Conversation History:")
        print("-"*80)
        if conversation_history:
            # Display all stored messages
            for msg in conversation_history:
                role_emoji = "👤" if msg['role'] == 'user' else "🤖"
                print(f"{role_emoji} {msg['role'].upper()}: {msg['message']}\n")
        else:
            print("No history yet.\n")
        print("-"*80 + "\n")
        continue
    
    # ========================================
    # VALIDATION: Empty input
    # ========================================
    elif not user_input:
        print("⚠️  Please type something.\n")
        continue
    
    # ========================================
    # MAIN: Process question with RAG + Memory
    # ========================================
    try:
        # Call the RAG function
        answer = ask_question(user_input)
        
        # Display formatted response
        print(f"\n🤖 Bot: {answer}\n")
        print("-"*80 + "\n")
        
    except Exception as e:
        # Handle any errors gracefully
        print(f"\n❌ Error: {e}\n")
        print("Please try asking your question differently.\n")


🎉 TAJ MAHAL CHATBOT
Ask me anything about the Taj Mahal!
Commands: quit | clear | history


🤖 Bot: The Taj Mahal was commissioned by Shah Jahan, the fifth Mughal emperor of India, in 1631. It was built as a mausoleum for his beloved wife, Mumtaz Mahal, and also houses Shah Jahan's tomb. The construction was overseen by a board of architects led by Ustad Ahmad Lahori.

--------------------------------------------------------------------------------


🤖 Bot: The Taj Mahal is special for several reasons:

1. **Architectural Masterpiece**: It is a prime example of Mughal architecture, combining elements from Persian, Islamic, and Indian styles. Its intricate design, use of precious stones, and symmetrical layout showcase the artistry of the time.

2. **Symbol of Love**: Commissioned by Shah Jahan in memory of his beloved wife Mumtaz Mahal, the Taj Mahal is widely regarded as a symbol of eternal love and devotion.

3. **UNESCO World Heritage Site**: Designated in 1983, it is recognized for

---
## Part 9: Testing the Chatbot

### Recommended Test Questions

Test the chatbot with these questions to see RAG + Memory in action:

**1. Basic Retrieval (No Memory Required):**
```
Q: "Who built the Taj Mahal?"
Expected: Bot searches vector DB and finds "Shah Jahan"
```

**2. Follow-up Using Memory:**
```
Q: "Why did he build it?"
Expected: Bot uses memory to know "he" = Shah Jahan
         Searches for motivation/reason
```

**3. New Topic (Fresh Retrieval):**
```
Q: "What materials were used in construction?"
Expected: Bot searches for construction materials
```

**4. Follow-up with Reference:**
```
Q: "Where did those materials come from?"
Expected: Bot remembers "those materials" from previous Q
         Searches for material origins
```

**5. Complex Memory Test:**
```
Q: "When was it built?"
Q: "How long did the construction take?"
Q: "Was it expensive?" ← Requires remembering "it" = Taj Mahal
```

**6. Clear and Reset:**
```
Command: clear
Q: "Why was it built?" ← Without memory, might need more context
```

### What You'll Learn

- **RAG in action**: See how vector search retrieves relevant chunks
- **Memory persistence**: Notice how follow-up questions work seamlessly
- **Context tracking**: Observe pronoun resolution (he, it, that, etc.)
- **Sliding window**: Ask 6+ questions to see oldest memories fade
- **Error handling**: Try edge cases and see graceful fallbacks

### Pro Tips

1. **Use `history` command** after a few exchanges to see memory structure
2. **Test without `clear`** to see how context builds up
3. **Ask ambiguous follow-ups** to test memory quality
4. **Compare with/without memory** by clearing mid-conversation

---
## Summary: What You've Built

### Architecture Overview

```
┌─────────────────────────────────────────────────────────────┐
│                    USER QUESTION                            │
└─────────────────────┬───────────────────────────────────────┘
                      │
        ┌─────────────┴─────────────┐
        │                           │
        ▼                           ▼
┌───────────────┐          ┌────────────────┐
│   RETRIEVER   │          │     MEMORY     │
│  (LangChain)  │          │   (Custom)     │
└───────┬───────┘          └────────┬───────┘
        │                           │
        │  Top 4 Chunks             │  Previous Q&A
        │                           │
        └─────────┬─────────────────┘
                  │
                  ▼
        ┌─────────────────┐
        │  Format Prompt  │
        │  instructions + │
        │     input       │
        └────────┬────────┘
                 │
                 ▼
        ┌─────────────────┐
        │  OpenAI API     │
        │ responses.create│
        └────────┬────────┘
                 │
                 ▼
        ┌─────────────────┐
        │    RESPONSE     │
        │  Update Memory  │
        └─────────────────┘
```

### Components Used

**LangChain (Retrieval Layer):**
- `WebBaseLoader`: Scrape web content
- `RecursiveCharacterTextSplitter`: Chunk documents
- `OpenAIEmbeddings`: Generate embeddings
- `Chroma`: Vector database with persistence
- `Retriever`: Semantic search interface

**Direct OpenAI API (Generation Layer):**
- `client.responses.create()`: Generate answers
- `instructions`: System prompt + memory
- `input`: Retrieved context + question
- `output_text`: Clean string response

**Custom Memory (Context Management):**
- `conversation_history`: List of dicts
- `add_to_memory()`: Save messages
- `get_memory_text()`: Format for API
- Sliding window: 10 messages max

### Key Learnings

1. **Hybrid Architecture**: Combine LangChain's strengths (retrieval) with direct API control (generation)
2. **RAG Pipeline**: Retrieve → Augment → Generate workflow
3. **Memory Management**: Manual tracking enables flexible conversation flow
4. **Vector Search**: Semantic similarity finds relevant content, not just keywords
5. **Prompt Engineering**: Separate context (input) from instructions (system prompt)

### Next Steps

- **Add more sources**: Scrape multiple URLs for broader knowledge
- **Improve chunking**: Experiment with chunk size and overlap
- **Better memory**: Implement summarization for longer conversations
- **Add citations**: Return source documents with answers
- **Deploy**: Build a web interface with Streamlit or Gradio